In [ ]:
import os
llama2_13b = "meta/llama-2-13b-chat:f4e2de70d66816a838a89eeeb621910adffb0dd0baba3976c96980970978018d"
os.environ["REPLICATE_API_TOKEN"] = ""

In [ ]:
import pandas as pd

df = pd.read_csv('drugLibTrain_raw.tsv', delimiter="\t")
drugs  = sorted(df["urlDrugName"].unique())

tmp = df.groupby(["condition"])["condition"].count().sort_values(ascending=False).head(50).to_frame()
tmp.columns = ['count']

conditions  = sorted(tmp.reset_index()['condition'])

df2=pd.read_csv('drugs_for_common_treatments.csv')

In [ ]:

def md(t):
  display(Markdown(t))

In [ ]:
import replicate
# langchain setup
from langchain.llms import Replicate
# Use the Llama 2 model hosted on Replicate
# Temperature: Adjusts randomness of outputs, greater than 1 is random and 0 is deterministic, 0.75 is a good starting value
# top_p: When decoding text, samples from the top p percentage of most likely tokens; lower to ignore less likely tokens
# max_new_tokens: Maximum number of tokens to generate. A word is generally 2-3 tokens
llama_model = Replicate(
    model=llama2_13b,
    model_kwargs={"temperature": 0.5,"top_p": 1, "max_new_tokens":500}
)

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
loaded_documents = []

for DATA_PATH in['drugLibTrain_raw.tsv','drugLibTest_raw.tsv']:
    loader = CSVLoader(file_path=DATA_PATH, csv_args={'delimiter': '\t'},source_column="condition")
    documents = loader.load()
    loaded_documents.extend(documents)

#DATA_PATH = 'drugs_for_common_treatments.csv'
#loader = CSVLoader(file_path=DATA_PATH,source_column="drug_name")
#documents = loader.load()
#loaded_documents.extend(documents)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
splits = text_splitter.split_documents(loaded_documents)

model_name='sentence-transformers/all-MiniLM-L6-v2'
#model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name,
                                       model_kwargs={'device': 'cpu'})

#DB_FAISS_PATH = 'vectorstore/db_faiss'
vectorstore = FAISS.from_documents(splits, embeddings)
#db.save_local(DB_FAISS_PATH)

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import HumanMessagePromptTemplate,SystemMessagePromptTemplate,ChatPromptTemplate

## 1 Provide the summary of public reviews per drug

In [ ]:
#1serious
general_system_template1 = r"""
Act as a medical assistant, given the drug name - please provide the summary of all the reviews,
highlighting the percent of negative and positive reviews.
Please wish health and provide one fun proverb at the end.
 ----
{context}
----
"""
general_user_template1 = "Question:```{question}```"
messages1 = [
            SystemMessagePromptTemplate.from_template(general_system_template1),
            HumanMessagePromptTemplate.from_template(general_user_template1)
]
qa_prompt1 = ChatPromptTemplate.from_messages( messages1)
chain1 = ConversationalRetrievalChain.from_llm(llama_model, vectorstore.as_retriever(), return_source_documents=True,
                                             combine_docs_chain_kwargs={"prompt": qa_prompt1})

In [ ]:
#1playful
general_system_template2 = r"""
Act as a friend: calm me down and given the drug name - please provide the summary of all the reviews,
in form of a funny story like I am a child.
Please wish health and provide one fun proverb at the end.
 ----
{context}
----
"""
general_user_template2 = "Question:```{question}```"
messages2 = [
            SystemMessagePromptTemplate.from_template(general_system_template2),
            HumanMessagePromptTemplate.from_template(general_user_template2)
]
qa_prompt2 = ChatPromptTemplate.from_messages( messages2)
chain2 = ConversationalRetrievalChain.from_llm(llama_model, vectorstore.as_retriever(), return_source_documents=True,
                                             combine_docs_chain_kwargs={"prompt": qa_prompt2})

llm_resp = {"serious" : chain1, "playful": chain2}

In [ ]:
def give_summary(chain,query,chat_history = []):
    #chat_history = [(query, result["answer"])] #if we need to make a chatbot
    result = chain({"question": query, "chat_history": chat_history})
    return(md(result['answer']))

In [ ]:
import ipywidgets as widgets

In [ ]:
drug_dropdown = widgets.Dropdown(
    options=drugs,
    value=drugs[0],
    description='Drug name:',
    disabled=False,
)
display(drug_dropdown)

llm_dropdown = widgets.Dropdown(
    options=list(llm_resp.keys()),
    value=list(llm_resp.keys())[0],
    description='Summary:',
    disabled=False,
)
display(llm_dropdown)

button = widgets.Button(
    description='Give summary',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run report',
    icon='check'
)
display(button)

out = widgets.Output(layout={'border': '1px solid black'})
display(out)

def on_button_clicked(b):
    with out:
        out.clear_output()
        give_summary(llm_resp[llm_dropdown.value],drug_dropdown.value)
        drug_url  = df2[df2['drug_name'].str.lower() == drug_dropdown.value.lower()]["drug_link"].head(1) #checking URL in the second dataset
        if not drug_url.empty:
            print("URL of the drug: ", drug_url.to_string(index=False, header=False))


button.on_click(on_button_clicked, False)

Dropdown(description='Drug name:', options=('abilify', 'accolate', 'accupril', 'accutane', 'aciphex', 'actiq',…

Dropdown(description='Summary:', options=('serious', 'playful'), value='serious')

Button(button_style='info', description='Give summary', icon='check', style=ButtonStyle(), tooltip='Run report…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

## 2 Compare public reviews per 2 drugs

In [ ]:
#1serious
general_system_template2 = r"""
Act as a medical assistant, given the 2 drug names separated by comma - compare  all the reviews,
highlighting the percent of negative and positive reviews and sideeffects.
At the end advice to consult with a health professional when choosing the best drug for you.
 ----
{context}
----
"""
general_user_template2 = "Question:```{question}```"
messages2 = [
            SystemMessagePromptTemplate.from_template(general_system_template2),
            HumanMessagePromptTemplate.from_template(general_user_template2)
]
qa_prompt2 = ChatPromptTemplate.from_messages( messages2)
chain2 = ConversationalRetrievalChain.from_llm(llama_model, vectorstore.as_retriever(), return_source_documents=True,
                                             combine_docs_chain_kwargs={"prompt": qa_prompt2})

In [ ]:
#1playful
general_system_template3 = r"""
Act as a friend, given the 2 drug names separated by comma  - tell me a funny story about them
helping me to better understand the posisitve and negative effects of these drugs on other people.
At the end advice to consult with a health professional when choosing the best drug for you.
 ----
{context}
----
"""
general_user_template3 = "Question:```{question}```"
messages3 = [
            SystemMessagePromptTemplate.from_template(general_system_template3),
            HumanMessagePromptTemplate.from_template(general_user_template3)
]
qa_prompt3 = ChatPromptTemplate.from_messages( messages3)
chain3 = ConversationalRetrievalChain.from_llm(llama_model, vectorstore.as_retriever(), return_source_documents=True,
                                             combine_docs_chain_kwargs={"prompt": qa_prompt3})

llm_resp1 = {"serious" : chain2, "funny story": chain3}

In [ ]:
def compare_reviews(chain,drug1,drug2,chat_history = []):
    query = f"{drug1}, {drug2}"
    result = chain({"question": query, "chat_history": chat_history})
    return(md(result['answer']))

In [ ]:
drug_dropdown1 = widgets.Dropdown(
    options=drugs,
    value=drugs[0],
    description='Drug name:',
    disabled=False,
)
display(drug_dropdown1)

drug_dropdown2 = widgets.Dropdown(
    options=drugs,
    value=drugs[0],
    description='Drug name:',
    disabled=False,
)
display(drug_dropdown2)

llm_dropdown1 = widgets.Dropdown(
    options=list(llm_resp1.keys()),
    value=list(llm_resp1.keys())[0],
    description='Response:',
    disabled=False,
)
display(llm_dropdown1)

button1 = widgets.Button(
    description='Compare reviews',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run report',
    icon='check'
)
display(button1)

out1 = widgets.Output(layout={'border': '1px solid black'})
display(out1)

def on_button_clicked1(b):
    with out1:
        out1.clear_output()
        compare_reviews(llm_resp1[llm_dropdown1.value],drug_dropdown1.value,drug_dropdown2.value)


button1.on_click(on_button_clicked1, False)

Dropdown(description='Drug name:', options=('abilify', 'accolate', 'accupril', 'accutane', 'aciphex', 'actiq',…

Dropdown(description='Drug name:', options=('abilify', 'accolate', 'accupril', 'accutane', 'aciphex', 'actiq',…

Dropdown(description='Response:', options=('serious', 'funny story'), value='serious')

Button(button_style='info', description='Compare reviews', icon='check', style=ButtonStyle(), tooltip='Run rep…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

## 2 List top 3 drugs for given condition and summary of their reviews

In [ ]:
#1serious
general_system_template3 = r"""
Act as a medical assistant, given the condition - show top 3 drugs for this condition according to the user reviews.
Show negative and positive side effects and advice to consult to professional at the end.

 ----
{context}
----
"""
general_user_template3 = "Question:```{question}```"
messages3 = [
            SystemMessagePromptTemplate.from_template(general_system_template3),
            HumanMessagePromptTemplate.from_template(general_user_template3)
]
qa_prompt3 = ChatPromptTemplate.from_messages( messages3)
chain3 = ConversationalRetrievalChain.from_llm(llama_model, vectorstore.as_retriever(), return_source_documents=True,
                                             combine_docs_chain_kwargs={"prompt": qa_prompt3})

In [ ]:
#1game
general_system_template4 = r"""
Act as a friend, given the condition - create a fun game scenario teaching about the top 3 drugs for this condition.
Advice to consult to professional at the end.

 ----
{context}
----
"""
general_user_template4 = "Question:```{question}```"
messages4 = [
            SystemMessagePromptTemplate.from_template(general_system_template4),
            HumanMessagePromptTemplate.from_template(general_user_template4)
]
qa_prompt4 = ChatPromptTemplate.from_messages( messages4)
chain4 = ConversationalRetrievalChain.from_llm(llama_model, vectorstore.as_retriever(), return_source_documents=True,
                                             combine_docs_chain_kwargs={"prompt": qa_prompt4})

llm_resp2 = {"serious" : chain3, "game": chain4}

In [ ]:
def drug_for_condition(chain,condition,chat_history = []):
    query = condition
    result = chain({"question": query, "chat_history": chat_history})
    return(md(result['answer']))

In [ ]:
condition_dropdown = widgets.Dropdown(
    options=conditions,
    value=conditions[0],
    description='Condition:',
    disabled=False,
)
display(condition_dropdown)

llm_dropdown2 = widgets.Dropdown(
    options=list(llm_resp2.keys()),
    value=list(llm_resp2.keys())[0],
    description='Top 3 drugs:',
    disabled=False,
)
display(llm_dropdown2)

button2 = widgets.Button(
    description='Give top 3 drugs',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Run report',
    icon='check'
)
display(button2)

out2 = widgets.Output(layout={'border': '1px solid black'})
display(out2)

def on_button_clicked2(b):
    with out2:
        out2.clear_output()
        drug_for_condition(llm_resp2[llm_dropdown2.value],condition_dropdown.value)


button2.on_click(on_button_clicked2, False)

Dropdown(description='Condition:', options=('acid reflux', 'acne', 'add', 'adhd', 'adult acne', 'allergies', '…

Dropdown(description='Top 3 drugs:', options=('serious', 'game'), value='serious')

Button(button_style='info', description='Give top 3 drugs', icon='check', style=ButtonStyle(), tooltip='Run re…

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…